## 0. 前準備

Poetry (python の仮想環境)をインストール

```
brew install poetry
```

### 0.1 graphene-studio のフォルダー内で、仮想環境を生成

```
poetry install
```

### 0.2 VSCode で、test.ipynb を開き、2 で生成した python kernel を指定して実行


## 1. 外場なしの配置

外場は与えず、原子間斥力のみ与えて Quench して原子配置を均一化する。


In [ ]:
from logging import getLogger

import numpy as np

from graphenestudio.quench import quench_particles
from graphenestudio.pack import random_box
from graphenestudio.interaction import repulsive_potential, repulsive_force

logger = getLogger()

# 斥力で配置を調整する原子の個数。最終的に配置する炭素の数ではない。
Natom = 192
# 立方体セル
cell = np.diag(np.array([5.0, 5.0, 5.0]))

# 初期配置。セル相対。
r = random_box(Natom)

# ポテンシャルエネルギーは粒子位置の関数。
pot = lambda r, cell: repulsive_potential(r, cell, repul=4, rc=2.0)
# その勾配。
dpot = lambda r, cell: -repulsive_force(r, cell, repul=4, rc=2.0)

r_quenched = quench_particles(r, cell, pot, dpot)
x_quenched = r_quenched @ cell

Quench 前とあとを比較。


In [ ]:
# -*- coding: utf-8 -*-
import plotly.graph_objects as go
from itertools import starmap

x = r @ cell

# mode='makers'を指定しないと点の間に線が引かれる
fig = go.Figure(
    data=[
        go.Scatter3d(
            x=x[:, 0],
            y=x[:, 1],
            z=x[:, 2],
            mode="markers",
            marker=dict(
                size=5,
                # color=point_sum,
                colorscale="Viridis",
                # symbol=result_symbol
            ),
        ),
        go.Scatter3d(
            x=x_quenched[:, 0],
            y=x_quenched[:, 1],
            z=x_quenched[:, 2],
            mode="markers",
            marker=dict(
                size=5,
                # color=point_sum,
                colorscale="Viridis",
                # symbol=result_symbol
            ),
        ),
    ]
)
fig.show()

## 2. graphene の本格的生成

外場を加えたエネルギー関数とその傾きの関数を準備する。


In [ ]:
# The gyroid surface and its gradient
from logging import getLogger, basicConfig, INFO

import numpy as np

from graphenestudio.pack import random_box
from graphenestudio.interaction import repulsive_potential, repulsive_force

from graphenestudio.surface import Gyroid, Sphere, Diamond, PSurface

# from graphenestudio.surface.psurface import PSurface


basicConfig(level=INFO)
logger = getLogger()

# 斥力粒子の個数
Natom = 540
# 外場の強度。小さいと、ぎゅうぎゅうづめになった時に面外に粒子がこぼれる可能性がある。
cost = 100000
# 立方体セル
cell = np.diag(np.array([5.0, 5.0, 5.0]))

# 曲面関数
# surf = Gyroid(eccentricity=0.0)
# surf = Sphere(radius=0.45)
surf = Diamond(eccentricity=0.0)
# surf = PSurface(eccentricity=0.0)

# 斥力粒子の座標(セル相対)
r = random_box(Natom)

# 斥力粒子のポテンシャルエネルギー関数
pot = lambda r, cell: repulsive_potential(
    r, cell, repul=4, rc=1.0
) + cost * surf.exfield_potential(r)
# その勾配
dpot = lambda r, cell: -repulsive_force(
    r, cell, repul=4, rc=1.0
) + cost * surf.exfield_gradient(r)

### 2.1 Quench し、表示する。


In [ ]:
from graphenestudio.quench import quench_particles
import plotly.graph_objects as go

r_quenched = quench_particles(r, cell, pot, dpot)
x_quenched = r_quenched @ cell


# mode='makers'を指定しないと点の間に線が引かれる
fig = go.Figure(
    data=[
        # go.Scatter3d(
        #     x=x[:, 0],
        #     y=x[:, 1],
        #     z=x[:, 2],
        #     mode="markers",
        #     marker=dict(
        #         size=6,
        #         # color=point_sum,
        #         colorscale="Viridis",
        #         # symbol=result_symbol
        #     ),
        # ),
        go.Scatter3d(
            x=x_quenched[:, 0],
            y=x_quenched[:, 1],
            z=x_quenched[:, 2],
            mode="markers",
            marker=dict(
                size=5,
                # color=point_sum,
                colorscale="Viridis",
                # symbol=result_symbol
            ),
        ),
    ]
)
fig.show()

### 2.2

Quench した構造に有限温度を与えてすこし原子が動けるようにし、100 ステップの疑似分子動力学法を実施する。これにより、より歪みが少ない形状にたどりつけるはず。

斥力粒子を均質に配置して三角格子をつくったら、三角格子の重心同士を連結して、双対グラフを生成する。これがグラフェンの構造となる。一連の処理は graphenate 関数の内部で行われる。

できた構造を x, y, z 方向に 2 倍して 2x2x2 の構造を作り、さらにセルを z 方向に 2 倍する。

結果を以下のファイルに書きだす。

1. `.gro` 原子の位置
2. `.top` 結合
3. `.yap` 構造の preview


In [ ]:
# import importlib

# graphenestudio = importlib.import_module("graphenestudio")

from graphenestudio import (
    draw_yaplot,
    graphenate,
    dump_gro,
    generate_top,
    replicate_x,
    replicate_y,
    replicate_z,
    extend_z,
)


base = "diamond"
with open(f"{base}.yap", "w") as file:
    count = 100
    for x, cell, g in graphenate(
        Natom,
        cell,
        pot,
        dpot,
        dt=0.0005,  # 0.005
        T=0.1,
    ):

        # x, cell, g = replicate_x(x, cell, g)
        # x, cell, g = replicate_y(x, cell, g)
        # x, cell, g = replicate_z(x, cell, g)
        x, cell, g = extend_z(x, cell, g)

        print(count)
        file.write(draw_yaplot(x, cell, g))
        with open(f"{base}_{count}.itp", "w") as ft:
            ft.write(generate_top(x, cell, g, generate_dihed_list=True))
        with open(f"{base}_{count}.gro", "w") as fg:
            dump_gro(x, cell, g, fg)

        count -= 1
        if count == 0:
            break

In [ ]:
# CIFに変換し、あとはVESTAにまかせるか。

import MDAnalysis as mda

u = mda.Universe("diamond_100.gro")  # , 'graphene.top')
ag = u.select_atoms("name C")
ag.write("diamond_100.pdb")

## 2.3 py3Dmol でとりあえず表示


In [ ]:
import py3Dmol

# show
view = py3Dmol.view()
view.addModel(open("diamond_100.gro").read(), "gro")
view.setStyle({"stick": {}})
view.addUnitCell()
view.zoomTo()
view.show()

以下、rindo 研究には使わない。

## 拡張版

曲面を解析的な式ではなく、グリッド上の数値配列として与える。

`cube`や`sphere`は 3 次元グリッドで定義される$w=f(x,y,z)$型の空間関数で、これが$f(x,y,z)=0$を横切る isosurface で、graphene の形状を指定する。


In [ ]:
from logging import INFO, basicConfig, getLogger, DEBUG

import numpy as np

from graphenestudio.quench import quench_particles
from graphenestudio.pack import random_box
from graphenestudio.interaction import repulsive_potential, repulsive_force
from graphenestudio.surface import Grid, Ticks, GridSurfaceFunc


basicConfig(level=INFO)
logger = getLogger()

Natom = 192
cost = 10000
cell = np.diag(np.array([5.0, 5.0, 5.0]))

r = 0.4  # "radius" of the cube, i.e., half length of an edge

XYZ = np.mgrid[-0.5:0.5:11j, -0.5:0.5:11j, -0.5:0.5:11j]
xticks = Ticks(min=-0.5, binw=0.1)
yticks = Ticks(min=-0.5, binw=0.1)
zticks = Ticks(min=-0.5, binw=0.1)

# グリッドタイプの関数定義法
cube = Grid(
    values=np.max(np.abs(XYZ), axis=0) - r, xticks=xticks, yticks=yticks, zticks=zticks
)

sphere = Grid(
    values=np.sum(XYZ**2, axis=0) - r * r,
    xticks=xticks,
    yticks=yticks,
    zticks=zticks,
)

surf = GridSurfaceFunc(sphere)

In [ ]:
r = random_box(Natom)
# x = r @ cell
pot = lambda r, cell: repulsive_potential(
    r, cell, repul=4, rc=1.0
) + cost * surf.exfield_potential(r)
dpot = lambda r, cell: -repulsive_force(
    r, cell, repul=4, rc=1.0
) + cost * surf.exfield_gradient(r)

r_quenched = quench_particles(r, cell, pot, dpot)
x_quenched = r_quenched @ cell

In [ ]:
# -*- coding: utf-8 -*-
import plotly.graph_objects as go


# mode='makers'を指定しないと点の間に線が引かれる
fig = go.Figure(
    data=[
        # go.Scatter3d(
        #     x=x[:, 0],
        #     y=x[:, 1],
        #     z=x[:, 2],
        #     mode="markers",
        #     marker=dict(
        #         size=6,
        #         # color=point_sum,
        #         colorscale="Viridis",
        #         # symbol=result_symbol
        #     ),
        # ),
        go.Scatter3d(
            x=x_quenched[:, 0],
            y=x_quenched[:, 1],
            z=x_quenched[:, 2],
            mode="markers",
            marker=dict(
                size=5,
                # color=point_sum,
                colorscale="Viridis",
                # symbol=result_symbol
            ),
        ),
    ]
)
fig.show()

x2top が生成した top ファイルの中身を確認する。


In [ ]:
import networkx as nx

sections = dict()
name = None
with open("gyroid.top") as f:
    for line in f:
        line = line.strip()
        if len(line) > 0 and line[0] == "[":
            name = line
            sections[name] = []
            continue
        if name is not None:
            sections[name].append(line)

# make a graph based on bonds
g0 = nx.Graph()
for line in sections["[ bonds ]"]:
    if len(line) > 0 and line[0] != ";":
        cols = line.split()
        g0.add_edge(int(cols[0]), int(cols[1]))

print(g0.degree)


print(len(sections["[ bonds ]"]))
print(len(sections["[ pairs ]"]))
# split a multiframed gro file into gro files.

import gromacs

with open("gyroid99.gro") as fr:
    for i, frame in enumerate(gromacs.read_gro(fr)):
        break
    g = nx.Graph(
        [
            (i, j)
            for i, j in pl.pairs_iter(
                frame["position"],
                0.18,
                frame["cell"],
                fractional=False,
                distance=False,
            )
        ]
    )